# --- Day 11: Cosmic Expansion ---
https://adventofcode.com/2023/day/11

## --- Part One ---

In [261]:
import numpy as np
import math

base_galaxy_1d = []
base_galaxy_width = 0
base_galaxy_height = 0

with open("input.txt", "r") as f:
# with open("sample.txt", "r") as f:
    lines = f.read().splitlines()
    base_galaxy_width = len(lines[0])
    base_galaxy_height = len(lines)
    count = 1
    for line in lines:
        for char in line:
            if char == "#":
                base_galaxy_1d.append(count)
                count += 1
            else:
                base_galaxy_1d.append(char)


# converts a 1d galaxy into a 2d galaxy
def get2dgalaxy(galaxy1d, gWidth, gHeight):
    return np.reshape(galaxy1d, (gHeight, gWidth))


# expanded the galaxy
def expand_2d_universe(galaxy):
    # universe is square so every i counts as row and column
    to_expand = [[], []]
    expanded_galaxy = []
    for i in range(0, np.shape(galaxy)[0]):
        row = galaxy[i, :]
        col = galaxy[:, i]
        if np.count_nonzero(row == ".") == base_galaxy_width:
            to_expand[0].append(i)
        if np.count_nonzero(col == ".") == base_galaxy_height:
            to_expand[1].append(i)
    for i, row in enumerate(galaxy):
        expanded_galaxy.append(list(row))
        if (i) in to_expand[0]:
            expanded_galaxy.append(list(row))
    for idx, row in enumerate(expanded_galaxy):
        count = 0
        for item in to_expand[1]:
            expanded_galaxy[idx].insert(item + count, ".")
            count += 1
    return np.array(expanded_galaxy)


base_galaxy_2d = get2dgalaxy(base_galaxy_1d, base_galaxy_width, base_galaxy_height)
expanded_galaxy_2d = expand_2d_universe(base_galaxy_2d)
expanded_galaxy_1d = expanded_galaxy_2d.flatten()
# init vars to hold new width and height
expanded_galaxy_height = len(expanded_galaxy_2d)
expanded_galaxy_width = len(expanded_galaxy_2d[0])

locations1d = {}

for i, v in enumerate(expanded_galaxy_1d):
    if v.isdigit():
        locations1d[int(v)] = i

def get2dlocation(location1d):
    return [math.floor(location1d/expanded_galaxy_width), location1d%expanded_galaxy_width]


def get_distance(loc1, loc2):
    loc1xy = get2dlocation(loc1)
    loc2xy = get2dlocation(loc2)
    return abs(loc1xy[0]-loc2xy[0])+abs(loc1xy[1]-loc2xy[1])

scores = []
cnt = len(locations1d.keys())+1
for i in locations1d:
    for x in range(i,cnt):
        if x == i:
            continue
        dist = get_distance(locations1d[i], locations1d[x])
        scores.append(dist)

print("The answer to part 1 is:", sum(scores))


The answer to part 1 is: 9609130


## --- Part Two ---

In [260]:
import numpy as np
import math
import copy

space = []
space_width = 0
space_height = 0
baseGalaxyCoords = {}

with open("input.txt", "r") as f:
# with open("sample.txt", "r") as f:
    lines = f.read().splitlines()
    space_width = len(lines[0])
    space_height = len(lines)
    count = 1
    for line in lines:
        for char in line:
            if char == "#":
                space.append(count)
                count += 1
            else:
                space.append(char)

# get 2d coord location based on 1d
def getbase2dlocation(loc1d):
    return [math.floor(loc1d/space_width), loc1d%space_width]

# find the original coords of each planet
def getBaseGalaxyCoord(space):
    out = {}
    for i, v in enumerate(space):
        if str(v).isdigit():
            out[v] = getbase2dlocation(i)
    return out

baseGalaxyCoords = getBaseGalaxyCoord(space)

def getExpansionDimens(space):
    # reshape space into 2d array
    space2d = np.reshape(space, (space_height, space_width))
    # universe is square so every i counts as row and column
    dimens = [[], []]
    for i in range(0, np.shape(space2d)[0]):
        row = space2d[i, :]
        col = space2d[:, i]
        if np.count_nonzero(row == ".") == space_width:
            dimens[0].append(i)
        if np.count_nonzero(col == ".") == space_height:
            dimens[1].append(i)
    return dimens

expansionDimens = getExpansionDimens(space)

# calc new coords
def getUpdatedCoords(baseGalaxyCoords, expansionDimens, expConstant):
    newGalaxyCoords = copy.deepcopy(baseGalaxyCoords)
    # new approach , we loop through all the entries in the dict 
    for galaxy in newGalaxyCoords:
        # we sum up the amount of items in [row,col] that are below the threshold
        xval = sum([x<baseGalaxyCoords[galaxy][1] for x in expansionDimens[1]])
        yval = sum([y<baseGalaxyCoords[galaxy][0] for y in expansionDimens[0]])
        # we then update those values based on the sum*(expansion constant-1)
        newGalaxyCoords[galaxy][1] += xval*(expConstant-1)
        newGalaxyCoords[galaxy][0] += yval*(expConstant-1)
    # return the updated coords
    return newGalaxyCoords
            
newGalaxyCoords = getUpdatedCoords(baseGalaxyCoords, expansionDimens, 1000000)

# calculate distance between 2 galaxies
def get_distance(loc1, loc2):
    return abs(loc1[0]-loc2[0])+abs(loc1[1]-loc2[1])

# keep track of scores
scores = []
# keep counter
cnt = len(newGalaxyCoords.keys())+1
for i in newGalaxyCoords:
    for x in range(i,cnt):
        # only look for values higher than what it's currently checking
        if x == i:
            continue
        # find the distance
        dist = get_distance(newGalaxyCoords[i], newGalaxyCoords[x])
        # append the distance to the list
        scores.append(dist)

# print the sum of all distances
print("The answer to part 2 is:", sum(scores))

The answer to part 2 is: 702152204842
